# Better Way 26. 믹스인 유틸리티 클래스에만 다중 상속을 사용하자

다중 상속: 단어 그대로 하나의 클래스가 여러 개의 클래스를 상속하는 것
- 일반적으로 다중 상속은 권장되지 않음. (Better Way 25 참고)
- 따라서, 다중상속의 편리함과 캡슐화가 필요하다면 **믹스인(mix-in)** 작성하자!
    - 믹스인: 클래스에서 제공해야 하는 추가적인 메서드만 정의하는 작은 클래스.
    - 자체 인스턴스 속성(attribute) 정의하지 않고, 
    - 생성자를 호출하도록 요구하지 않음.

In [1]:
'''
예제: 파이썬 객체를 메모리 내부 표현에서 직렬화(serialization)용 딕셔너리로 변환하는 기능 구현
--> 모든 클래스에서 사용할 수 있게 범용으로 작성
'''
class ToDictMixin(object):
    def to_dict(self):
        return self._traverse_dict(self.__dict__)
    
    def _traverse_dict(self, instance_dict):
        output = {}
        for key, value in instance_dict.items():
            output[key] = self._traverse(key, value)
        return output
    
    def _traverse(self, key, value):
        if isinstance(value, ToDictMixin):
            return value.to_dict()
        elif isinstance(value, dict):
            return self._traverse_dict(value)
        elif isinstance(value, list):
            return [self._traverse(key, i) for i in value]
        elif hasattr(value, '__dict__'):
            return self._traverse_dict(value.__dict__)
        else:
            return value

In [2]:
class BinaryTree(ToDictMixin):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [3]:
tree = BinaryTree(10,
                 left=BinaryTree(7, right=BinaryTree(9)),
                 right=BinaryTree(13, left=BinaryTree(11)))
print(tree.to_dict())

{'left': {'left': None, 'right': {'left': None, 'right': None, 'value': 9}, 'value': 7}, 'right': {'left': {'left': None, 'right': None, 'value': 11}, 'right': None, 'value': 13}, 'value': 10}


In [4]:
'''
예제: 부모 노드에 대한 참조를 저장하는 BinaryTree의 서브클래스
--> 순환 참조는 ToDictMixin.to_dict의 구현이 무한 루프에 빠지게 만든다.
'''
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent

In [5]:
root = BinaryTreeWithParent(10) #{value: 10, left:None, right:None, parent:None}
root.left = BinaryTreeWithParent(7, parent=root) #parent를 참조하는 과정에서 자기 자신(root.left) 참조하는 루프 발생
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

RecursionError: maximum recursion depth exceeded

In [6]:
'''
예제: 부모 노드에 대한 참조를 저장하는 BinaryTree의 서브클래스
--> 순환 참조는 ToDictMixin.to_dict의 구현이 무한 루프에 빠지게 만든다.
--> _traverse 메서드를 오버라이드하여 부모의 숫자 값만 꺼내오게 만든다.
'''
class BinaryTreeWithParent(BinaryTree):
    def __init__(self, value, left=None, right=None, parent=None):
        super().__init__(value, left=left, right=right)
        self.parent = parent
        
    def _traverse(self, key, value):
        if (isinstance(value, BinaryTreeWithParent) and key == 'parent'):
            return value.value
        else:
            return super()._traverse(key, value)

In [7]:
root = BinaryTreeWithParent(10)
root.left = BinaryTreeWithParent(7, parent=root)
root.left.right = BinaryTreeWithParent(9, parent=root.left)
print(root.to_dict())

{'left': {'left': None, 'parent': 10, 'right': {'left': None, 'parent': 7, 'right': None, 'value': 9}, 'value': 7}, 'parent': None, 'right': None, 'value': 10}


In [8]:
'''
BinaryTreeWithParent._traverse가 정의되어, 해당 타입의 속성이 있는 클래스라면 무엇이든 자동으로 ToDictMixin으로 동작 가능
'''
class NamedSubTree(ToDictMixin):
    def __init__(self, name, tree_with_parent):
        self.name = name
        self.tree_with_parent = tree_with_parent

In [10]:
my_tree = NamedSubTree('foobar', root.left.right)
print(my_tree.to_dict())

{'name': 'foobar', 'tree_with_parent': {'left': None, 'parent': 7, 'right': None, 'value': 9}}


In [16]:
'''
믹스인을 조합할 수 있음 --> 예시: JSON 직렬화를 제공하는 믹스인이 필요, 클래스에 to_dict 메서드가 있다고 가정하고 만든다
'''
import json

class JsonMixin(object):
    @classmethod
    def from_json(cls, data):
        kwargs = json.loads(data)
        return cls(**kwargs)
    
    def to_json(self):
        return json.dumps(self.to_dict())

위 예제에서 JsonMixin 클래스는 인스턴스 메서드와 클래스 메서드를 둘 다 정의하고 있음
- 클래스에 to_dict 메서드가 있고,
- 해당 클래스의 init 메서드에서 키워드 인수를 받는다는 것

In [17]:
class DatacenterRack(ToDictMixin, JsonMixin):
    def __init__(self, switch=None, machines=None):
        self.switch = Switch(**switch)
        self.machines = [Machine(**kwargs) for kwargs in machines]

class Switch(ToDictMixin, JsonMixin):
    def __init__(self, ports, speed):
        self.ports = ports
        self.speed = speed
    
class Machine(ToDictMixin, JsonMixin):
    def __init__(self, cores, ram, disk):
        self.cores = cores
        self.ram = ram
        self.disk = disk

In [21]:
# machine_spec = [(8, 32e9, 5e12), (4, 16e9, 1e12), (2, 4e9, 500e9)]
# switch = Switch(ports=5, speed=1e9)
# machines = [Machines(cores=c, ram=r, disk=d) for c, r, d in zip()]
serialized = """{
    "switch": {"ports": 5, "speed": 1e9},
    "machines": [
        {"cores": 8, "ram": 32e9, "disk": 5e12},
        {"cores": 4, "ram": 16e9, "disk": 1e12},
        {"cores": 2, "ram": 4e9, "disk": 500e9}
    ]
}"""

In [22]:
deserialized = DatacenterRack.from_json(serialized)
roundtrip = deserialized.to_json()
assert json.loads(serialized) == json.loads(roundtrip)

핵심 정리
- 믹스인 클래스로 같은 결과를 얻을 수 있다면 다중 상속을 사용하지 말자.
- 인스턴스 수준에서 동작을 교체할 수 있게 만들어서 믹스인 클래스가 요구할 때 클래스별로 원하는 동작을 하게 하자.
- 간단한 동작들로 복잡한 기능을 생성하려면 믹스인을 조합하자.

# Better Way 27. 공개 속성보다는 비공개 속성을 사용하자

In [23]:
'''
파이썬에서는 클래스 속성이 public과 private 두 유형으로만 정의된다
'''
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10 #private 속성은 이름 앞에 밑줄 두 개를 붙여 지정함
        
    def get_private_field(self):
        return self.__private_field  

In [24]:
foo = MyObject()
assert foo.public_field == 5 #public 속성은 어디서든 객체에 점 연산자(.) 사용하여 접근 가능

In [25]:
assert foo.get_private_field() == 10 #private 속성은 같은 클래스에 속한 메서드 내에서 직접 접근 가능
foo.__private_field #그러나 클래스 외부에서 직접 접근하면 예외 발생

AttributeError: 'MyObject' object has no attribute '__private_field'

In [26]:
class MyOtherObject(object):
    def __init__(self):
        self.__private_field = 71
    
    @classmethod
    def get_private_field_of_instance(cls, instance): #클래스 메서드에서도 같은 class 블록 내이므로 private 접근 가능
        return instance.__private_field

In [27]:
bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

In [31]:
class MyParentObject(object):
    def __init__(self):
        self.__private_field = 71
        
class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field #서브클래스에서는 부모 클래스의 private 속성에 접근 불가

In [32]:
baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

private 속성의 동작 --> 간단하게 속성 이름을 변환하는 방식으로 구현
- 먼저 \_MyChildObject__private_field에 접근하는 코드로 변환하고,
- MyParentObject.\_\_init__에만 정의되므로, \_MyParentObject__private_field이 됨
- 즉, 속성 이름이 변환되므로 (일치하지 않아서) 접근이 실패하는 것임.

In [33]:
assert baz._MyParentObject__private_field == 71 #그래서 이름만 제대로 넣으면 접근 가능함!
print(baz.__dict__)

{'_MyParentObject__private_field': 71}


파이썬의 private 속성용 문법: (위에서 보다시피) 가시성(visibility)을 **엄격하게 강제하지 않는다**
1. 개방 > 폐쇄
2. 언어 기능을 가로채는 기능(BetterWay 32 참고)이 있으면, 언제든지 객체의 내부를 조작할 수 있음
==> 무분별하게 객체 내부에 접근하는 위험을 최소화하기 위하여, PEP 8 (BetterWay 2 참고) 스타일을 따른다.
\_protected_field처럼 앞에 밑줄을 하나 추가하여 클래스의 외부 사용자들이 신중하게 다뤄야 함을 보여줌.

In [53]:
class MyClass(object):
    def __init__(self, value):
        self.__value = value #서브클래스나 외부에서 접근하면 안되는 내부 API를 private 속성으로 정의 (실수!!)
        
    def get_value(self):
        return str(self.__value) 

In [54]:
foo = MyClass(5)
assert foo.get_value() == '5'

In [55]:
class MyIntegerSubClass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

In [56]:
foo = MyIntegerSubClass(5)
assert foo.get_value() == 5

In [57]:
class MyBaseClass(object): #새로운 최상위 클래스가 추가되어 클래스 계층 구조가 변경
    def __init__(self, value):
        self.__value = value
    
    def get_value(self):
        return self.__value
        
class MyClass(MyBaseClass):
    def get_value(self):
        return str(self.__value)
    
class MyIntegerSubClass(MyClass):
    def get_value(self):
        return int(self._MyClass__value) #더이상 이 이름으로 접근할 수 없음

In [58]:
foo = MyIntegerSubClass(5)
foo.get_value()

AttributeError: 'MyIntegerSubClass' object has no attribute '_MyClass__value'

- 일반적으로 protected 속성을 사용해서 서브클래스가 더 많은 일을 할 수 있게 하는 편이 낫다.
- 각각의 protected 필드를 문서화하여 서브클래스에서 내부 API 중 어느 것을 쓸 수 있고 어느 것을 그대로 둬야 하는지 설명한다.

In [ ]:
class MyClass(object):
    def __init__(self, value):
        # 사용자가 객체에 전달한 값을 저장한다.
        # 문자열로 강제할 수 있는 값이어야 하며,
        # 객체에 할당하고 나면 불변으로 취급해야 한다.
        self._value = value

- private 속성을 사용할지 고민하는 경우는 서브클래스와 이름이 충돌할 염려가 있을 때뿐이다.
- 예컨데, 자식 클래스가 부모 클래스에서 이미 정의한 속성을 정의할 때 일어나는 문제가 있다.

In [59]:
class ApiClass(object):
    def __init__(self):
        self._value = 5
        
    def get(self):
        return self._value


class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' #충돌 발생!!

In [60]:
a = Child()
print(a.get(), 'and', a._value, 'should be different')

hello and hello should be different


- 주로 클래스가 공개 API의 일부일 때 문제가 된다.
- 서브클래스는 (부모 클래스를) 직접 제어할 수 없으니 문제를 고치려고 리팩토링할 수 없다.
- 위와 같이 속성 이름이 아주 일반적인 경우 일어날 확률이 높다.

In [61]:
class ApiClass(object):
    def __init__(self):
        self.__value = 5
        
    def get(self):
        return self.__value #부모클래스에서 private 속성을 사용하여 자식 클래스와 속성 이름이 겹치지 않게 함.
    

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'

In [62]:
a = Child()
print(a.get(), 'and', a._value, 'should be different')

5 and hello should be different


핵심정리
- 파이썬 컴파일러는 private 속성을 엄격하게 강요하지 않는다.
- 서브클래스가 내부 API와 속성에 접근하지 못하게 막기보다는 처음부터 내부 API와 속성으로 더 많은 일을 할 수 있게 설계하자.
- private 속성에 대한 접근을 강제로 제어하지 말고 protected 필드를 문서화하여 서브클래스에 필요한 지침을 제공하자.
- 직접 제어할 수 없는 서브클래스와 이름이 충돌하지 않게 할 때만 private 속성을 사용하는 방안을 고려하자.

# Better Way 28. 커스텀 컨테이너 타입은 collections.abc의 클래스를 상속받게 만들자

파이썬 프로그래밍: 데이터를 담은 클래스들을 정의하고 이 객체들이 연계되는 방법을 명시하는 일
- 모든 파이썬 클래스는 일종의 컨테이너, 속성/기능을 함께 캡슐화
- 데이터 관리용 내장 컨테이너 타입(리스트, 튜플, 딕셔너리, 세트)

In [63]:
'''
예시: 멤버의 빈도를 세는 메서드를 추가로 갖춘 커스텀 리스트 타입 생성
'''
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)
        
    def frequency(self):
        counts ={}
        for item in self:
            counts.setdefault(item, 0) #key가 있으면 값을 반환, 없으면 0을 넣음
            counts[item] += 1
        return counts

In [64]:
foo = FrequencyList(['a','b','a','c','b','a','d'])
print('Length is', len(foo))
foo.pop() #마지막 element를 반환하고 삭제함
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

Length is 7
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


In [65]:
'''
예제: Binary Tree 클래스에 list, tuple과 같은 시퀀스 시맨틱 (인덱스로 접근하는 기능 등) 제공하고자 함
'''
class BinaryNode(object):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

In [ ]:
class IndexableNode(BinaryNode):
    def _search(self, count, index):
        # (found, count) 반환
    
    def __getitem__(self, index):
        found, _ = self._search(0, index)
        if not found:
            raise IndexError('Index out of range')
        return found.value

In [ ]:
class SequenceNode(IndexableNode):
    def __len__(self):
        _, count = self._search(0, None)
        return count

In [66]:
from collections.abc import Sequence #각 컨테이너 타입에 필요한 일반적인 메서드를 모두 제공하는 abstract class 정의함

class BadType(Sequence):
    pass

foo = BadType() #필수 메서드를 구현하지 않았으므로 에러 발생!

TypeError: Can't instantiate abstract class BadType with abstract methods __getitem__, __len__

In [67]:
'''
SequenceNode가 이미 요구하는 메서드를 모두 구현했으므로 별도 작업 없이
index, count 같은 부가적인 메서드를 모두 제공함
'''
class BetterNode(SequenceNode, Sequence):
    pass

NameError: name 'SequenceNode' is not defined

핵심 정리
- 쓰임새가 간단할 때는 list, dict 같은 파이썬의 컨테이너 타입에서 직접 상속받게 하자.
- 커스텀 컨테이너 타입을 올바르게 구현하는 데 필요한 많은 메서드에 주의해야 한다.
- 커스텀 컨테이너 타입이 collections.abc에 정의된 인터페이스에서 상속받게 만들어서 클래스가 필요한 인터페이스, 동작과 일치하게 하자.